In [ ]:
# PIPE DREAMS: Run this and subsequent PIPE DREAMS cells to get a viz of df nodes
from telco_churn.lineage_auto import LINEAGE


In [ ]:
from telco_churn.lineage_auto import LINEAGE

dq_graph = LINEAGE.build_graph(rankdir="LR")
dq_graph


In [ ]:
# # df lineage tracker | complex def
# from telco_churn.utils.lineage import DFLineageTracker
# from pathlib import Path

# # 2.L0 📊 Initialize DF lineage tracker for Section 2
# LINEAGE_PATH_2 = SEC2_REPORTS_DIR / "df_lineage_section2.csv"

# df_lineage = DFLineageTracker(
#     name="Section 2 Data Quality",
#     tracked_columns=["TotalCharges", "tenure", "MonthlyCharges"],
#     save_path=LINEAGE_PATH_2,
# )

# # First snapshot: initial load of df
# df_lineage.snapshot(
#     df=df,
#     section="2.0.0",
#     label="Initial load from RAW_DATA",
#     note="Raw Telco CSV as loaded in 2.0.0")
# 🔁 Simple DF lineage recorder for Section 2 (no new defs required if you don't want them)

if "df_lineage_records" not in globals():
    df_lineage_records = []

def _log_df_state(section, label, frame, note=None):
    """
    Tiny helper: record where df/df_clean is in the pipeline.
    (OK to keep this for Level_3; later you can move it to a library.)
    """
    rec = {
        "section": section,                # "2.0.0", "2.1.3", "2.6.7", etc.
        "label": label,                    # short human label
        "n_rows": int(frame.shape[0]),
        "n_cols": int(frame.shape[1]),
        "has_TotalCharges": "TotalCharges" in frame.columns,
        "TotalCharges_nulls": (
            int(frame["TotalCharges"].isna().sum())
            if "TotalCharges" in frame.columns else None
        ),
        "timestamp": pd.Timestamp.utcnow(),
        "note": note or "",
    }
    df_lineage_records.append(rec)


In [ ]:
# PIPE DREAMS AGG
# After 2.0.0 bootstrap / load
LINEAGE.register_step(
    globals(),
    step="2.0.0",
    description="Bootstrap & load raw Telco data"
)

# After 2.1.x schema normalization
LINEAGE.register_step(
    globals(),
    step="2.1.5",
    description="Post core numeric/categorical type alignment"
)

# After 2.3.14 numeric/categorical diagnostics
LINEAGE.register_step(
    globals(),
    step="2.3.14",
    description="Diagnostics complete – ready for controlled cleaning"
)

# 2.6.0 – right after you create df_before_clean / df_clean
LINEAGE.register_step(
    globals(),
    step="2.6.0",
    description="Snapshot before cleaning & initial df_clean"
)

# After a big cleaning step (say, missing value imputation)
LINEAGE.register_step(
    globals(),
    step="2.6.3",
    description="After missing-value imputation rules"
)

# After full 2.6.x completed
LINEAGE.register_step(
    globals(),
    step="2.6.12",
    description="Final cleaned df + audit metadata"
)


In [ ]:
lineage_df = LINEAGE.to_dataframe().sort_values("timestamp")

# High-level evolution:
display(
    lineage_df[
        ["step", "df_name", "n_rows", "n_cols", "cols_added", "cols_removed", "description"]
    ]
)


In [ ]:
df_clean_hist = (
    lineage_df[lineage_df["df_name"] == "df_clean"]
    .sort_values("timestamp")
)

display(df_clean_hist[["step", "n_rows", "n_cols", "cols_added", "cols_removed"]])


In [ ]:
from telco_churn.lineage_auto import LINEAGE

dq_graph = LINEAGE.build_graph(rankdir="LR")  # LR = left→right pipes
dq_graph
